K2 ALEATÓRIO PARA UMA ORDEM DE VARIÁVEIS

In [91]:
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import K2Score, BayesianEstimator
import pandas as pd
import random

def k2(dataset, parents_nmax):
    variables = list(dataset.columns)
    print(f'Ordem das variáveis iniciais: {variables}')
    random_order = random.sample(variables, len(variables))
    print(f'Ordem das variáveis aleatórias: {random_order}')
    estimator = K2Score(dataset)
    model = BayesianNetwork()
    nodes = list(random_order)  # nome dos nos
    model.add_nodes_from(nodes)

    for i in range(1, len(nodes)):
        node = nodes[i]  # no atual
        previous_nodes = nodes[:i]  # nos anteriores ao no atual
        parents = []  # pais do no atual
        P_old = estimator.local_score(node, parents)
        proceed = True

        while proceed and (len(parents) < parents_nmax):
            candidates = list(set(previous_nodes) - set(parents))  # nos candidatos para pais do no atual
            P_new = P_old  # new probability

            for candidate in candidates:
                candidate_score = estimator.local_score(node, parents + [candidate])  # pontuacao dos nos pais

                if candidate_score > P_new:
                    candidates_best = candidate  # melhor candidato para no pai
                    P_new = candidate_score

            if P_new > P_old:
                P_old = P_new
                parents.append(candidates_best)
                model.add_edge(candidates_best, node)

            else:
                proceed = False
    md = {'Estrutura do Modelo': list(model.edges)}
    score = {'Score': estimator.score(model)}
    return md, model, score

# Função para encontrar as CPDs
def tabular_cpd(model, data):
    for column in data.columns:
        data[column] = pd.Categorical(data[column])
    estimator = BayesianEstimator(model, data)
    cpds = [estimator.estimate_cpd(node) for node in model.nodes]
    return cpds

data = pd.read_csv('contact-lenses.csv')
best_model = k2(data, 4)
cpds = tabular_cpd(best_model[1], data)

print(f'Modelo: {best_model}')


Ordem das variáveis iniciais: ['age', 'spectacle-prescrip', 'astigmatism', 'tear-prod-rate', 'contact-lenses']
Ordem das variáveis aleatórias: ['spectacle-prescrip', 'contact-lenses', 'age', 'tear-prod-rate', 'astigmatism']
Modelo: ({'Estrutura do Modelo': [('contact-lenses', 'tear-prod-rate'), ('contact-lenses', 'astigmatism')]}, <pgmpy.models.BayesianNetwork.BayesianNetwork object at 0x000001A1BEAD94F0>, {'Score': -98.72503647411926})


GERAÇÃO DE 10 MODELOS DE FORMA ALEATÓRIA

In [67]:
from pgmpy.readwrite import XMLBIFWriter
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import K2Score, BayesianEstimator
import pandas as pd
import random
import time

def k2(dataset, parents_nmax, num_iterations):
    variables = list(dataset.columns)
    with open('hepartwo_aleat.txt', 'w') as file:  # Abra um arquivo de texto para escrita
        file.write(f'Ordem das variáveis iniciais: {variables}\n')
        
        estimator = K2Score(dataset)
        models_and_scores = []
        best_model = None
        best_score = float('-inf')
        best_order = None
        total_score = 0

        for iteration in range(num_iterations):
            random_order = random.sample(variables, len(variables))
            file.write(f'\nOrdem das variáveis aleatórias (iteração {iteration+1}): {random_order}\n')
            print(f'Ordem das variáveis aleatórias (iteração {iteration+1}): {random_order}')
            model = BayesianNetwork()
            model.add_nodes_from(random_order)

            for i in range(1, len(random_order)):
                node = random_order[i]  # no atual
                previous_nodes = random_order[:i]  # nos anteriores ao no atual
                parents = []  # pais do no atual
                P_old = estimator.local_score(node, parents)
                proceed = True

                while proceed and (len(parents) < parents_nmax):
                    candidates = list(set(previous_nodes) - set(parents))  # nos candidatos para pais do no atual
                    P_new = P_old  # new probability

                    for candidate in candidates:
                        candidate_score = estimator.local_score(node, parents + [candidate])  # pontuacao dos nos pais

                        if candidate_score > P_new:
                            candidates_best = candidate  # melhor candidato para no pai
                            P_new = candidate_score

                    if P_new > P_old:
                        P_old = P_new
                        parents.append(candidates_best)
                        model.add_edge(candidates_best, node)

                    else:
                        proceed = False

            score = estimator.score(model)
            total_score += score

            if score > best_score:
                best_model = model
                best_score = score
                best_order = random_order

            models_and_scores.append((model, score))
            file.write(f"Iteração {iteration + 1} - Score: {score}\n")
            print(f"Iteração {iteration + 1} - Score: {score}")
            file.write(f'Estrutura: {model.edges}\n')
            print(f'Estrutura: {model.edges}')
            file.write('------------------------------------------------------------------------------------------------')
            print('------------------------------------------------------------------------------------------------')

        avg_score = total_score / num_iterations

        file.write("\nMelhor Ordem, Estrutura e Score:\n")
        file.write(f"Ordem: {best_order}\n")
        file.write(f"Estrutura: {best_model.edges}\n")
        file.write(f"Score: {best_score}\n")
        file.write(f"Média dos Scores: {avg_score}\n")
        print("\nMelhor Ordem, Estrutura e Score:")
        print(f"Ordem: {best_order}")
        print(f"Estrutura: {best_model.edges}")
        print(f"Score: {best_score}")
        print(f"Média dos Scores: {avg_score}")

        return models_and_scores, best_model

# Função para encontrar as CPDs
def tabular_cpd(model, data):
    for column in data.columns:
        data[column] = pd.Categorical(data[column])
    estimator = BayesianEstimator(model, data)
    cpds = [estimator.estimate_cpd(node) for node in model.nodes]
    return cpds

data = pd.read_csv('hepartwo.csv')
num_iterations = 10

start_time = time.time()
models_and_scores, best_model = k2(data, 4, num_iterations)
cpds = tabular_cpd(best_model, data)
end_time = time.time()

# Calcule o tempo decorrido
elapsed_time = end_time - start_time

print(f"Tempo decorrido: {elapsed_time} segundos")
with open('hepartwo_aleat.txt', 'a') as file:  # Abrir o arquivo para atualizar
        file.write(f'Tempo gasto: {elapsed_time}\n')

# Adicione as CPDs ao modelo
for node, cpd in zip(best_model.nodes, cpds):
    best_model.add_cpds(cpd)

# Salve o modelo em formato XMLBIF
bif_writer = XMLBIFWriter(best_model)
bif_writer.write_xmlbif('hepartwo_aleat.xmlbif')



Ordem das variáveis aleatórias (iteração 1): ['density', 'nausea', 'pain_ruq', 'upper_pain', 'THepatitis', 'skin', 'spleen', 'proteins', 'cholesterol', 'consciousness', 'Hyperbilirubinemia', 'ama', 'albumin', 'surgery', 'injections', 'inr', 'ast', 'Cirrhosis', 'triglycerides', 'hepatomegaly', 'pressure_ruq', 'ChHepatitis', 'encephalopathy', 'jaundice', 'fatigue', 'fat', 'irregular_liver', 'spiders', 'transfusion', 'alt', 'vh_amn', 'age', 'diabetes', 'sex', 'bilirubin', 'hbc_anti', 'pain', 'edge', 'carcinoma', 'joints', 'alcohol', 'ascites', 'ESR', 'hbeag', 'hospital', 'hepatotoxic', 'bleeding', 'hbsag_anti', 'palms', 'choledocholithotomy', 'urea', 'ggtp', 'phosphatase', 'edema', 'anorexia', 'amylase', 'platelet', 'fibrosis', 'flatulence', 'hcv_anti', 'RHepatitis', 'PBC', 'alcoholism', 'gallstones', 'hbsag', 'obesity', 'hepatalgia', 'le_cells', 'itching', 'Steatosis']
Iteração 1 - Score: -328161.10673347436
Estrutura: [('density', 'consciousness'), ('density', 'encephalopathy'), ('nause